In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pyvi import ViTokenizer, ViPosTagger #pip install underthesea


In [2]:
# # Đọc file dữ liệu
# def read_file(filename):	    
#     filename=os.path.join(Path, filename)        
#     with open(filename, 'r',encoding="utf-8") as f:
#         data = f.read()

#     return data.encode('utf-8')
# Đọc file dữ liệu
def read_file(filename):
    filename = os.path.join(Path, filename)
    with open(filename, 'r', encoding="utf-8") as f:
        data = f.read()  # Đọc dữ liệu như một chuỗi

    return data  # Trả về chuỗi mà không mã hóa lại thành bytes


# Chuẫn hóa dữ liệu
def create_dataframe(matrix, tokens):
    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

# Tiền xử lý văn bản
import re
def Text_Preprocessing(doc):
    text_pre = doc.lower()
    text_pre=text_pre.replace("\n","")    
    text_pre=text_pre.lower() # Chuyển văn bản thành chữ thường    
    text_pre = re.sub(r"http(s)?://\S+", "", text_pre) # Loại bỏ URL
    text_pre=re.sub(r'[^\w\s]','',text_pre) # Xóa dấu câu
    text_pre = re.sub("\d+", " ", text_pre) # Xóa chữ số
    text_pre = re.sub(r"[!@#$[]()]'", "", text_pre) # Xoá kí tự đặc biệt: !@#$[]()
    text_pre = ''.join(text_pre)

    # # Xóa các từ không có nghĩa (Stop Words);
    # text_pre = ViTokenizer.tokenize(text_pre) # Tách từ (Tokenizing)
        ##Remove stop words
    filename=os.path.join(r"D:\Phuong\KhaiPhaDuLieuWeb\Chương 5\vietnamese-stopwords.txt")
    f = open(filename, "r", encoding="utf-8")
    
    # #Get Stop words Dictionaries
    List_StopWords=f.read().split("/n")
    text_pre=" ".join(text for text in text_pre.split() if text not in List_StopWords)
    
    text_pre = ViTokenizer.tokenize(text_pre)

    return text_pre

<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
C:\Users\OS\AppData\Local\Temp\ipykernel_31736\3844235627.py:31: SyntaxWarning: invalid escape sequence '\d'
  text_pre = re.sub("\d+", " ", text_pre) # Xóa chữ số


In [3]:
# Thêm tất cả các từ trong tập văn bản vào một tập hợp
doc_ = []
for i in range(1,4):
    filestore = f'D:/Phuong/KhaiPhaDuLieuWeb/Chương 5/BTCh5/{i}'
    for file in os.listdir(filestore):
        text_pre =''
        with open(os.path.join(filestore, file), 'r', encoding='utf-16 LE') as f:
            text = f.read()
            text_pre += text
        doc_.append(text_pre)

doc_


['\ufeffVừa trở về Việt Nam sau chuyến lưu diễn 3 tuần tại Mỹ, ca sĩ Cẩm Vân đã dành cho PV Thanh Niên cuộc trò chuyện.\nBao lâu rồi chị mới được mời lưu diễn dài ngày tại Mỹ? Chị nói rằng chuyến đi của mình lần này rất ấn tượng, tại sao?\nHơn một năm rồi tôi mới có dịp lưu diễn tại Mỹ. So với các chuyến đi khác, chuyến đi lần này rất ấn tượng bởi lẽ tôi là ca sĩ trong nước duy nhất được mời qua thể hiện các ca khúc của cố nhạc sĩ Trịnh Công Sơn nhân kỷ niệm 5 năm ngày mất của anh.\nỞ Mỹ có những chương trình riêng tưởng niệm nhạc sĩ Trịnh Công Sơn không, thưa chị?\nCó chứ! Chính xác là tôi đã diễn 2 show vào ngày 1.4 và ngày 9.4. Đây là 2 show diễn đầu tiên tưởng niệm nhạc sĩ Trịnh Công Sơn do nhóm The Friends (gồm 11 nhạc sĩ, nhạc công và 15 ca sĩ) tổ chức. Chương trình được đầu tư, dàn dựng rất công phu.\nLà ca sĩ trong nước duy nhất được mời tham gia chương trình, cảm giác của chị thế nào?\nTôi rất vui khi được thể hiện các ca khúc của anh Sơn trong chuyến lưu diễn lần này. Chương 

In [4]:
len(doc_)  # Số lượng văn bản

60

In [5]:
# # Define the Data
# doc_1=read_file("1.txt")	#Crawl from: https://vi.wikipedia.org/wiki/X%E1%BB%AD_l%C3%BD_ng%C3%B4n_ng%E1%BB%AF_t%E1%BB%B1_nhi%C3%AAn
# doc_2=read_file("2.txt") #Crawl from: https://en.wikipedia.org/wiki/Manchester_United_F.C.
# doc_3=read_file("3.txt")
# doc_4=read_file("4.txt")#Crawl from: https://vi.wikipedia.org/wiki/Arsenal_F.C.
data = []
for doc in doc_:
    doc_pre=Text_Preprocessing(doc)
    data.append(doc_pre)

data = data

In [6]:
# Xây dựng vector TF-IDF
# data = [' '.join(doc) for doc in data]
CountVect = CountVectorizer()
vector_matrix = CountVect.fit_transform(data)
vector_matrix.shape

(60, 4164)

In [7]:
# Lấy tên các từ (features)
tokens = CountVect.get_feature_names_out()
# Tạo DataFrame từ vector_matrix
vector_df = create_dataframe(vector_matrix.toarray(), tokens)

# Tính toán độ tương đồng cosine giữa các tài liệu
cosine_similarity_matrix = cosine_similarity(vector_matrix)

# Tạo doc_names từ doc_1 đến doc_n
doc_names = [f'doc_{i+1}' for i in range(cosine_similarity_matrix.shape[0])]

# Tạo DataFrame từ ma trận cosine similarity
Similarity = create_dataframe(cosine_similarity_matrix, doc_names)

# In ra DataFrame tương đồng cosine
print(Similarity)

           doc_1     doc_2     doc_3     doc_4     doc_5     doc_6     doc_7  \
doc_1   1.000000  0.245677  0.356101  0.472476  0.421723  0.357147  0.278731   
doc_2   0.245677  1.000000  0.280939  0.245449  0.329764  0.285621  0.259446   
doc_3   0.356101  0.280939  1.000000  0.493986  0.481483  0.654793  0.385712   
doc_4   0.472476  0.245449  0.493986  1.000000  0.454350  0.443821  0.359337   
doc_5   0.421723  0.329764  0.481483  0.454350  1.000000  0.425473  0.377809   
doc_6   0.357147  0.285621  0.654793  0.443821  0.425473  1.000000  0.382472   
doc_7   0.278731  0.259446  0.385712  0.359337  0.377809  0.382472  1.000000   
doc_8   0.353038  0.173990  0.434937  0.422482  0.359270  0.391711  0.366743   
doc_9   0.370144  0.298947  0.378441  0.373218  0.544831  0.385859  0.312187   
doc_10  0.315752  0.309907  0.496917  0.417765  0.485355  0.491759  0.392803   
doc_11  0.231379  0.184490  0.185090  0.254102  0.432988  0.211307  0.170890   
doc_12  0.157788  0.075903  0.112091  0.

In [8]:
import pandas as pd

# Tạo DataFrame từ ma trận cosine similarity
Similarity_df = pd.DataFrame(Similarity, index=doc_names, columns=doc_names)

# Chuyển DataFrame thành dạng long-form
similarity_long = Similarity_df.stack().reset_index()
similarity_long.columns = ['doc_1', 'doc_2', 'Do_tuong_dong']

# Loại bỏ các dòng có doc_1 == doc_2 (để tránh độ tương đồng của tài liệu với chính nó)
similarity_long = similarity_long[similarity_long['doc_1'] != similarity_long['doc_2']]

# Sắp xếp theo độ tương đồng tăng dần
similarity_sorted = similarity_long.sort_values(by='Do_tuong_dong', ascending=False)

# In ra kết quả
print(similarity_sorted)


       doc_1   doc_2  Do_tuong_dong
1656  doc_28  doc_37       0.799138
2187  doc_37  doc_28       0.799138
1711  doc_29  doc_32       0.695100
1888  doc_32  doc_29       0.695100
132    doc_3  doc_13       0.680788
...      ...     ...            ...
2171  doc_37  doc_12       0.044115
708   doc_12  doc_49       0.041060
2891  doc_49  doc_12       0.041060
2411  doc_41  doc_12       0.036730
700   doc_12  doc_41       0.036730

[3540 rows x 3 columns]
